In [1]:
import dwave
import numpy as np
import openml
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import plotly.express as px
from plotly.subplots import make_subplots
import dimod
from dwave.system import LeapHybridCQMSampler

# dataset = openml.datasets.get_dataset(312)

clf = RandomForestClassifier()

In [2]:
df = pd.read_csv("Dataset-stock-VISA.csv")

In [3]:
df.head()

,Date,Close,High,Low,SMA5,SMA10,SMA20,SMA30,RSI,%K,...,Aroon Up 10,Aroon Down 10,Aroon Up 15,Aroon Down 15,Aroon Up 20,Aroon Down 20,Aroon Up 30,Aroon Down 30,Price Change,Class
0,2021-02-09,206.520004,207.630005,203.699997,206.558002,202.020001,202.984000,207.026999,50.599307,76.849829,...,40.0,80.0,26.666667,53.333333,95.0,40.0,90.000000,26.666667,0.080002,0
1,2021-02-10,206.440002,207.970001,205.940002,207.574002,203.167001,202.863000,206.820666,51.556517,76.401388,...,50.0,90.0,33.333333,60.000000,100.0,45.0,93.333333,30.000000,-4.220001,1
2,2021-02-11,210.660004,211.589996,207.190002,207.856003,204.411002,202.928500,206.697000,60.079327,95.047962,...,10.0,100.0,6.666667,66.666667,5.0,50.0,96.666667,33.333333,0.699997,0
3,2021-02-12,209.960007,211.130005,207.679993,208.094003,206.082002,203.333501,206.417000,60.559752,91.320608,...,20.0,100.0,13.333333,73.333333,10.0,55.0,100.000000,36.666667,2.060013,0
4,2021-02-16,207.899994,211.050003,207.350006,208.296002,207.036002,203.649001,206.056000,56.762341,80.351423,...,30.0,90.0,20.000000,80.000000,15.0,60.0,100.000000,40.000000,0.389999,0


In [4]:
df.drop('Price Change',axis=1,inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 473 entries, 0 to 472
Data columns (total 28 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           473 non-null    object 
 1   Close          473 non-null    float64
 2   High           473 non-null    float64
 3   Low            473 non-null    float64
 4   SMA5           473 non-null    float64
 5   SMA10          473 non-null    float64
 6   SMA20          473 non-null    float64
 7   SMA30          473 non-null    float64
 8   RSI            473 non-null    float64
 9   %K             473 non-null    float64
 10  %D             473 non-null    float64
 11  ATR            473 non-null    float64
 12  ADX            473 non-null    float64
 13  EMA5           473 non-null    float64
 14  EMA10          473 non-null    float64
 15  EMA20          473 non-null    float64
 16  EMA30          473 non-null    float64
 17  Aroon Up 5     473 non-null    float64
 18  Aroon Down

In [6]:
# from sklearn.datasets import load_breast_cancer
from dwave.plugins.sklearn import SelectFromQuadraticModel

X = df.drop(['Class','Date'], axis=1)
y = df['Class']


X.head()

,Close,High,Low,SMA5,SMA10,SMA20,SMA30,RSI,%K,%D,...,Aroon Up 5,Aroon Down 5,Aroon Up 10,Aroon Down 10,Aroon Up 15,Aroon Down 15,Aroon Up 20,Aroon Down 20,Aroon Up 30,Aroon Down 30
0,206.520004,207.630005,203.699997,206.558002,202.020001,202.984000,207.026999,50.599307,76.849829,82.640476,...,80.0,100.0,40.0,80.0,26.666667,53.333333,95.0,40.0,90.000000,26.666667
1,206.440002,207.970001,205.940002,207.574002,203.167001,202.863000,206.820666,51.556517,76.401388,77.391670,...,100.0,20.0,50.0,90.0,33.333333,60.000000,100.0,45.0,93.333333,30.000000
2,210.660004,211.589996,207.190002,207.856003,204.411002,202.928500,206.697000,60.079327,95.047962,82.766393,...,20.0,40.0,10.0,100.0,6.666667,66.666667,5.0,50.0,96.666667,33.333333
3,209.960007,211.130005,207.679993,208.094003,206.082002,203.333501,206.417000,60.559752,91.320608,87.589986,...,40.0,60.0,20.0,100.0,13.333333,73.333333,10.0,55.0,100.000000,36.666667
4,207.899994,211.050003,207.350006,208.296002,207.036002,203.649001,206.056000,56.762341,80.351423,88.906665,...,60.0,80.0,30.0,90.0,20.000000,80.000000,15.0,60.0,100.000000,40.000000


In [7]:
X_new = SelectFromQuadraticModel(num_features=5).fit_transform(X, y)


/home/naman/Documents/RASSE/ocean/lib/python3.11/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but SelectFromQuadraticModel was fitted without feature names
  warnings.warn(


In [8]:
print(X_new.shape)

X_new

(473, 5)


array([[  5.21428571,  43.44075026,  80.        ,  80.        ,
         90.        ],
       [  5.18642862,  41.06369203, 100.        ,  90.        ,
         93.33333333],
       [  5.30214255,  37.21812644,  20.        , 100.        ,
         96.66666667],
       ...,
       [  4.42642648,  15.83263083, 100.        ,  20.        ,
         43.33333333],
       [  4.37285614,  14.47579403, 100.        ,  30.        ,
         46.66666667],
       [  4.40428489,  13.91987282, 100.        ,  40.        ,
         50.        ]])

In [9]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score

In [10]:
def train_test_models(X, y):
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Create a list to store the results
    results = []
    
    # Models
    models = {
        'SVM': SVC(),
        'Decision Tree': DecisionTreeClassifier(),
        'Random Forest': RandomForestClassifier(),
        'KNN': KNeighborsClassifier(),
        'Logistic Regression': LogisticRegression(max_iter=1500),
        'Naive Bayes': GaussianNB(),
        'Gradient Boosting': GradientBoostingClassifier(),
        'XGBoost': xgb.XGBClassifier()
    }
    
    for model_name, model in models.items():
        # Train the model
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test)
        
        # Calculate accuracy and F1 score
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        
        # Store the results as a dictionary
        results.append({'Model': model_name, 'Accuracy': accuracy, 'F1 Score': f1})
    
    # Create a DataFrame from the results list
    results_df = pd.DataFrame(results)
    
    return results_df

In [11]:

# Train and test the models
results = train_test_models(X_new, y)

# Print the results
results.head(10)

,Model,Accuracy,F1 Score
0,SVM,0.631579,0.615385
1,Decision Tree,0.463158,0.370370
2,Random Forest,0.557895,0.533333
3,KNN,0.547368,0.505747
4,Logistic Regression,0.442105,0.464646
5,Naive Bayes,0.557895,0.543478
6,Gradient Boosting,0.526316,0.516129
7,XGBoost,0.484211,0.461538


In [12]:
X_new3 = SelectFromQuadraticModel(num_features=3).fit_transform(X, y)


/home/naman/Documents/RASSE/ocean/lib/python3.11/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but SelectFromQuadraticModel was fitted without feature names
  warnings.warn(


In [13]:
# Train and test the models
results = train_test_models(X_new3, y)

# Print the results
results.head(10)

,Model,Accuracy,F1 Score
0,SVM,0.578947,0.523810
1,Decision Tree,0.421053,0.367816
2,Random Forest,0.515789,0.510638
3,KNN,0.389474,0.369565
4,Logistic Regression,0.452632,0.469388
5,Naive Bayes,0.505263,0.494624
6,Gradient Boosting,0.494737,0.510204
7,XGBoost,0.484211,0.494845


In [14]:
X_new8 = SelectFromQuadraticModel(num_features=8).fit_transform(X, y)

/home/naman/Documents/RASSE/ocean/lib/python3.11/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but SelectFromQuadraticModel was fitted without feature names
  warnings.warn(


In [15]:
# Train and test the models
results = train_test_models(X_new8, y)

# Print the results
results.head(10)

,Model,Accuracy,F1 Score
0,SVM,0.547368,0.598131
1,Decision Tree,0.536842,0.541667
2,Random Forest,0.600000,0.586957
3,KNN,0.568421,0.568421
4,Logistic Regression,0.484211,0.542056
5,Naive Bayes,0.494737,0.489362
6,Gradient Boosting,0.494737,0.466667
7,XGBoost,0.536842,0.531915


In [16]:
num_cols = X_new.shape[1]
col_names = [f'col{i+1}' for i in range(num_cols)]
qa5 = pd.DataFrame(data=X_new, columns=col_names)

# Save the DataFrame to a CSV file
qa5.to_csv('QA-5-features-VISA.csv', index=False)

In [17]:
num_cols = X_new3.shape[1]
col_names = [f'col{i+1}' for i in range(num_cols)]
qa3 = pd.DataFrame(data=X_new3, columns=col_names)

# Save the DataFrame to a CSV file
qa3.to_csv('QA-3-features-VISA.csv', index=False)

In [18]:
num_cols = X_new8.shape[1]
col_names = [f'col{i+1}' for i in range(num_cols)]
qa8 = pd.DataFrame(data=X_new8, columns=col_names)

# Save the DataFrame to a CSV file
qa8.to_csv('QA-8-features=VISA.csv', index=False)